# Notebook Summary

Get the global overall data for the crypto market

* Get the L1 specific cryptodominance
* Get some overall statistics of the crypto market

## Things to do
* Allow for appends to update data
* Think of the index that should be used (esp the market cap pct table)
* Any way to make the code more efficient?
* Catch errors if results are not returning 

## Get Crypto Overall Performance

In [1]:
import pandas as pd
import requests
import os
import datetime
import numpy as np
from datetime import timezone
import time

url = 'https://api.coingecko.com/api/v3/'
saved_file_location = '../data/raw/'
file_name_mc_share = 'JY-Overall-MC-Share.csv'
file_name_overall_mc = 'JY-Overall-MC.csv'
file_name_defi = 'JY-Overall-DeFi.csv'
global_endpoint = 'global'
defi_endpoint = 'global/decentralized_finance_defi'

### Atomic Functions

In [2]:
# Store L1 Currencies market cap percentage
def get_raw_total_market_cap(url, global_endpoint):
    response = requests.get(url + global_endpoint)
    global_crypto_json = response.json()
    return global_crypto_json

def get_market_cap_share(global_crypto_json):
    market_cap_pct = pd.DataFrame(
        data=global_crypto_json['data']['market_cap_percentage'].values(),
        index=global_crypto_json['data']['market_cap_percentage'].keys(),
        columns=['market_cap_percentage']
    )

    market_cap_pct['timestamp'] = pd.to_datetime(global_crypto_json['data']['updated_at'], 
                                     unit='s')
    market_cap_pct['date'] = market_cap_pct['timestamp'].dt.date
    market_cap_pct['time'] = market_cap_pct['timestamp'].dt.time
    market_cap_pct = market_cap_pct.reset_index().rename({'index':'coin'},axis=1)
    
    return market_cap_pct

def get_total_mc(global_crypto_json):
    # Store total crypto market cap statistics
    total_market_cap_stats = pd.DataFrame(
    
    data = {'active_cryptocurrencies': global_crypto_json['data']['active_cryptocurrencies'],
            'markets':global_crypto_json['data']['markets'],
            'total_market_cap_usd':global_crypto_json['data']['total_market_cap']['usd'],
            'timestamp': pd.to_datetime(global_crypto_json['data']['updated_at'], unit='s')},
    
    index=[pd.to_datetime(global_crypto_json['data']['updated_at'], unit='s').date()])
    
    total_market_cap_stats['date'] = total_market_cap_stats['timestamp'].dt.date
    total_market_cap_stats['time'] = total_market_cap_stats['timestamp'].dt.time
    total_market_cap_stats = total_market_cap_stats.reset_index().drop('index',axis=1)
    
    return total_market_cap_stats

def get_defi_mc(url, defi_endpoint):
    response = requests.get(url + defi_endpoint)
    global_defi_json = response.json()

    defi_df = (pd.DataFrame(global_defi_json).T.reset_index().drop('index',axis=1))
    defi_df['ingestion_time'] = datetime.datetime.now(timezone.utc)
    defi_df['date'] = defi_df['ingestion_time'].dt.date
    defi_df['time'] = defi_df['ingestion_time'].dt.time
    defi_df = defi_df.astype({
        'defi_market_cap':'float64',
        'eth_market_cap':'float64',
        'defi_to_eth_ratio':'float64',
        'trading_volume_24h':'float64',
        'defi_dominance':'float64',
        'top_coin_defi_dominance':'float64'}).round(2)
    defi_df.round(3)
    
    return defi_df

### overall data ingestion functions

In [3]:
def overall_crypto_mc(url, global_endpoint, saved_file_location, file_name_mc_share, file_name_overall_mc):
    
    global_crypto_json = get_raw_total_market_cap(url, global_endpoint)
    print(f'CoinGecko ingestion time: {pd.to_datetime(global_crypto_json["data"]["updated_at"],unit="s")}')
    
    ### Crypto Market Share ###
    # Check if the file exists. If it doesn't, create the file and add data
    if os.path.isfile(saved_file_location+file_name_mc_share) is False:
        print(f'The file {file_name_mc_share} does not exist')
        market_cap_share = get_market_cap_share(global_crypto_json)
        market_cap_share.to_csv(saved_file_location+file_name_mc_share,index=False)
    
    # If it exists, append new data
    else:
        print(f'The file {file_name_mc_share} exists, appending new info')

        # Retrieve the details of the df
        target_df = pd.read_csv(saved_file_location+file_name_mc_share)
        print(f'Before ingestion, overall MC shape is {target_df.shape}.shape')

        # Retrieve the details of the new data
        market_cap_pct = get_market_cap_share(global_crypto_json)

        # Append the data
        target_df = pd.concat([target_df, market_cap_pct])
        print(f'After ingestion, overall MC shape is {target_df.shape}.shape')
        # Save the file
        target_df.to_csv(saved_file_location+file_name_mc_share,index=False)

        
    ### Crypto Overall Statistics ###
    # Check if the file exists. If it doesn't, create the file and add data
    if os.path.isfile(saved_file_location+file_name_overall_mc) is False:
        print(f'The file {file_name_overall_mc} does not exist')
        total_mc = get_total_mc(global_crypto_json)
        total_mc.to_csv(saved_file_location+file_name_overall_mc,index=False)
    # If it exists, append new data
    else:
        print(f'The file {file_name_overall_mc} exists, appending new info')

        # Retrieve the details of the df
        target_df = pd.read_csv(saved_file_location+file_name_overall_mc)
        print(f'Before ingestion, overall MC shape is {target_df.shape}')
        # Retrieve the details of the new data
        new_total_mc = get_total_mc(global_crypto_json)

        # Append the data
        target_df = pd.concat([target_df, new_total_mc])
        print(f'After ingestion, overall MC shape is {target_df.shape}')
        # Save the file
        target_df.to_csv(saved_file_location+file_name_overall_mc,index=False)

In [4]:
def overall_defi_mc(url, defi_endpoint, saved_file_location, file_name_defi):

    # Check if the file exists. If it doesn't, create the file and add data
    if os.path.isfile(saved_file_location+file_name_defi) is False:
        print(f'The file {file_name_defi} does not exist')
        overall_defi_df = get_defi_mc(url, defi_endpoint)
        overall_defi_df.to_csv(saved_file_location+file_name_defi,index=False)
    
    # If it exists, append new data
    else:
        print(f'The file {file_name_defi} exists, appending new info')

        # Retrieve the details of the df
        target_df = pd.read_csv(saved_file_location+file_name_defi)
        print(f'Before ingestion, overall defi shape is {target_df.shape}')
        # Get new data
        new_defi_df = get_defi_mc(url, defi_endpoint)

        # Append the data
        target_df = pd.concat([target_df, new_defi_df])
        print(f'After ingestion, overall defi shape is {target_df.shape}')
        # Save the file
        target_df.to_csv(saved_file_location+file_name_defi,index=False)
    

# 'Cron'

In [5]:
counter = 0
while True:
    counter += 1
    print(f'Data has been inserted {counter} times')
    print(f'Current Data ingestion time: {datetime.datetime.now(timezone.utc)}')
    overall_crypto_mc(url, global_endpoint, saved_file_location, file_name_mc_share, file_name_overall_mc)
    overall_defi_mc(url, defi_endpoint, saved_file_location, file_name_defi)
    time.sleep(780)


Data has been inserted 1 times
Current Data ingestion time: 2022-03-01 04:58:14.406523+00:00
CoinGecko ingestion time: 2022-03-01 04:52:16
The file JY-Overall-MC-Share.csv does not exist
The file JY-Overall-MC.csv does not exist
The file JY-Overall-DeFi.csv does not exist
Data has been inserted 2 times
Current Data ingestion time: 2022-03-01 05:11:15.360057+00:00
CoinGecko ingestion time: 2022-03-01 05:02:23
The file JY-Overall-MC-Share.csv exists, appending new info
Before ingestion, overall MC shape is (10, 5).shape
After ingestion, overall MC shape is (20, 5).shape
The file JY-Overall-MC.csv exists, appending new info
Before ingestion, overall MC shape is (1, 6)
After ingestion, overall MC shape is (2, 6)
The file JY-Overall-DeFi.csv exists, appending new info
Before ingestion, overall defi shape is (1, 10)
After ingestion, overall defi shape is (2, 10)
Data has been inserted 3 times
Current Data ingestion time: 2022-03-01 05:24:16.995314+00:00
CoinGecko ingestion time: 2022-03-01 0

After ingestion, overall defi shape is (15, 10)
Data has been inserted 16 times
Current Data ingestion time: 2022-03-01 08:13:38.831200+00:00
CoinGecko ingestion time: 2022-03-01 08:13:26
The file JY-Overall-MC-Share.csv exists, appending new info
Before ingestion, overall MC shape is (150, 5).shape
After ingestion, overall MC shape is (160, 5).shape
The file JY-Overall-MC.csv exists, appending new info
Before ingestion, overall MC shape is (15, 6)
After ingestion, overall MC shape is (16, 6)
The file JY-Overall-DeFi.csv exists, appending new info
Before ingestion, overall defi shape is (15, 10)
After ingestion, overall defi shape is (16, 10)
Data has been inserted 17 times
Current Data ingestion time: 2022-03-01 08:26:39.864684+00:00
CoinGecko ingestion time: 2022-03-01 08:23:29
The file JY-Overall-MC-Share.csv exists, appending new info
Before ingestion, overall MC shape is (160, 5).shape
After ingestion, overall MC shape is (170, 5).shape
The file JY-Overall-MC.csv exists, appending

After ingestion, overall defi shape is (29, 10)
Data has been inserted 30 times
Current Data ingestion time: 2022-03-01 12:27:20.167105+00:00
CoinGecko ingestion time: 2022-03-01 12:24:37
The file JY-Overall-MC-Share.csv exists, appending new info
Before ingestion, overall MC shape is (290, 5).shape
After ingestion, overall MC shape is (300, 5).shape
The file JY-Overall-MC.csv exists, appending new info
Before ingestion, overall MC shape is (29, 6)
After ingestion, overall MC shape is (30, 6)
The file JY-Overall-DeFi.csv exists, appending new info
Before ingestion, overall defi shape is (29, 10)
After ingestion, overall defi shape is (30, 10)
Data has been inserted 31 times
Current Data ingestion time: 2022-03-01 12:40:21.200404+00:00
CoinGecko ingestion time: 2022-03-01 12:34:39
The file JY-Overall-MC-Share.csv exists, appending new info
Before ingestion, overall MC shape is (300, 5).shape
After ingestion, overall MC shape is (310, 5).shape
The file JY-Overall-MC.csv exists, appending

After ingestion, overall defi shape is (43, 10)
Data has been inserted 44 times
Current Data ingestion time: 2022-03-01 15:29:40.881412+00:00
CoinGecko ingestion time: 2022-03-01 15:25:32
The file JY-Overall-MC-Share.csv exists, appending new info
Before ingestion, overall MC shape is (430, 5).shape
After ingestion, overall MC shape is (440, 5).shape
The file JY-Overall-MC.csv exists, appending new info
Before ingestion, overall MC shape is (43, 6)
After ingestion, overall MC shape is (44, 6)
The file JY-Overall-DeFi.csv exists, appending new info
Before ingestion, overall defi shape is (43, 10)
After ingestion, overall defi shape is (44, 10)
Data has been inserted 45 times
Current Data ingestion time: 2022-03-01 15:42:41.822304+00:00
CoinGecko ingestion time: 2022-03-01 15:35:36
The file JY-Overall-MC-Share.csv exists, appending new info
Before ingestion, overall MC shape is (440, 5).shape
After ingestion, overall MC shape is (450, 5).shape
The file JY-Overall-MC.csv exists, appending

After ingestion, overall defi shape is (57, 10)
Data has been inserted 58 times
Current Data ingestion time: 2022-03-01 18:31:56.575170+00:00
CoinGecko ingestion time: 2022-03-01 18:26:31
The file JY-Overall-MC-Share.csv exists, appending new info
Before ingestion, overall MC shape is (570, 5).shape
After ingestion, overall MC shape is (580, 5).shape
The file JY-Overall-MC.csv exists, appending new info
Before ingestion, overall MC shape is (57, 6)
After ingestion, overall MC shape is (58, 6)
The file JY-Overall-DeFi.csv exists, appending new info
Before ingestion, overall defi shape is (57, 10)
After ingestion, overall defi shape is (58, 10)
Data has been inserted 59 times
Current Data ingestion time: 2022-03-01 18:44:57.574713+00:00
CoinGecko ingestion time: 2022-03-01 18:36:33
The file JY-Overall-MC-Share.csv exists, appending new info
Before ingestion, overall MC shape is (580, 5).shape
After ingestion, overall MC shape is (590, 5).shape
The file JY-Overall-MC.csv exists, appending

...

After ingestion, overall defi shape is (99, 10)
Data has been inserted 100 times
Current Data ingestion time: 2022-03-02 03:38:47.871841+00:00
CoinGecko ingestion time: 2022-03-02 03:29:05
The file JY-Overall-MC-Share.csv exists, appending new info
Before ingestion, overall MC shape is (990, 5).shape
After ingestion, overall MC shape is (1000, 5).shape
The file JY-Overall-MC.csv exists, appending new info
Before ingestion, overall MC shape is (99, 6)
After ingestion, overall MC shape is (100, 6)
The file JY-Overall-DeFi.csv exists, appending new info
Before ingestion, overall defi shape is (99, 10)
After ingestion, overall defi shape is (100, 10)
Data has been inserted 101 times
Current Data ingestion time: 2022-03-02 03:51:48.736744+00:00
CoinGecko ingestion time: 2022-03-02 03:49:12
The file JY-Overall-MC-Share.csv exists, appending new info
Before ingestion, overall MC shape is (1000, 5).shape
After ingestion, overall MC shape is (1010, 5).shape
The file JY-Overall-MC.csv exists, ap

After ingestion, overall defi shape is (113, 10)
Data has been inserted 114 times
Current Data ingestion time: 2022-03-02 06:41:03.619432+00:00
CoinGecko ingestion time: 2022-03-02 06:39:59
The file JY-Overall-MC-Share.csv exists, appending new info
Before ingestion, overall MC shape is (1130, 5).shape
After ingestion, overall MC shape is (1140, 5).shape
The file JY-Overall-MC.csv exists, appending new info
Before ingestion, overall MC shape is (113, 6)
After ingestion, overall MC shape is (114, 6)
The file JY-Overall-DeFi.csv exists, appending new info
Before ingestion, overall defi shape is (113, 10)
After ingestion, overall defi shape is (114, 10)
Data has been inserted 115 times
Current Data ingestion time: 2022-03-02 06:54:04.549880+00:00
CoinGecko ingestion time: 2022-03-02 06:50:07
The file JY-Overall-MC-Share.csv exists, appending new info
Before ingestion, overall MC shape is (1140, 5).shape
After ingestion, overall MC shape is (1150, 5).shape
The file JY-Overall-MC.csv exists

After ingestion, overall defi shape is (127, 10)
Data has been inserted 128 times
Current Data ingestion time: 2022-03-02 09:43:25.452351+00:00
CoinGecko ingestion time: 2022-03-02 09:41:12
The file JY-Overall-MC-Share.csv exists, appending new info
Before ingestion, overall MC shape is (1270, 5).shape
After ingestion, overall MC shape is (1280, 5).shape
The file JY-Overall-MC.csv exists, appending new info
Before ingestion, overall MC shape is (127, 6)
After ingestion, overall MC shape is (128, 6)
The file JY-Overall-DeFi.csv exists, appending new info
Before ingestion, overall defi shape is (127, 10)
After ingestion, overall defi shape is (128, 10)
Data has been inserted 129 times
Current Data ingestion time: 2022-03-02 09:56:26.751786+00:00
CoinGecko ingestion time: 2022-03-02 09:51:19
The file JY-Overall-MC-Share.csv exists, appending new info
Before ingestion, overall MC shape is (1280, 5).shape
After ingestion, overall MC shape is (1290, 5).shape
The file JY-Overall-MC.csv exists

After ingestion, overall defi shape is (141, 10)
Data has been inserted 142 times
Current Data ingestion time: 2022-03-02 12:45:44.050131+00:00
CoinGecko ingestion time: 2022-03-02 12:42:22
The file JY-Overall-MC-Share.csv exists, appending new info
Before ingestion, overall MC shape is (1410, 5).shape
After ingestion, overall MC shape is (1420, 5).shape
The file JY-Overall-MC.csv exists, appending new info
Before ingestion, overall MC shape is (141, 6)
After ingestion, overall MC shape is (142, 6)
The file JY-Overall-DeFi.csv exists, appending new info
Before ingestion, overall defi shape is (141, 10)
After ingestion, overall defi shape is (142, 10)
Data has been inserted 143 times
Current Data ingestion time: 2022-03-02 12:58:45.065438+00:00
CoinGecko ingestion time: 2022-03-02 12:52:25
The file JY-Overall-MC-Share.csv exists, appending new info
Before ingestion, overall MC shape is (1420, 5).shape
After ingestion, overall MC shape is (1430, 5).shape
The file JY-Overall-MC.csv exists

KeyboardInterrupt: 